<a href="https://colab.research.google.com/github/prasannashrestha011/PL_prediction_2025-26-season/blob/main/Laliga_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [202]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [203]:

data_path="/content/drive/MyDrive/data/laliga"
os.listdir(data_path)
file_path=os.path.join(data_path,'LaLiga_Matches.csv')


In [204]:
import os
import pandas as pd
import matplotlib as plt
import numpy as np
from collections import defaultdict
from typing import Dict,List,Tuple

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [205]:
df=pd.read_csv(file_path)
df.head()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR
0,1995-96,02-09-1995,La Coruna,Valencia,3,0,H,2.0,0.0,H
1,1995-96,02-09-1995,Sp Gijon,Albacete,3,0,H,3.0,0.0,H
2,1995-96,03-09-1995,Ath Bilbao,Santander,4,0,H,2.0,0.0,H
3,1995-96,03-09-1995,Ath Madrid,Sociedad,4,1,H,1.0,1.0,D
4,1995-96,03-09-1995,Celta,Compostela,0,1,A,0.0,0.0,D


In [206]:
df['HTHG']=df['HTHG'].fillna(np.median(df['HTHG']))
df['HTAG']=df['HTAG'].fillna(np.median(df['HTAG']))
df['HTR']=df['HTR'].fillna(df['HTR'].mode()[0])

df=df.drop(columns=['Date'])

In [207]:

season_matches = [
    (season, season_df)
    for season, season_df in df.groupby('Season')
    if season != '2023-24'
]


In [208]:
def summarise_season(matches:pd.DataFrame)->pd.DataFrame:
  teams:Dict[str,Dict[str,int]]=defaultdict(lambda:{
      "points":0,
      "wins":0,
      "losses":0,
      "draws":0,
      "goals_for":0,
      "goals_against":0,
      "goal_diff":0
  })
  for row,match in matches.iterrows():
      home,away=match['HomeTeam'],match['AwayTeam']
      hg,ag=match['FTHG'],match['FTAG']

      teams[home]["goals_for"]+=hg
      teams[home]["goals_against"]+=ag

      teams[away]["goals_for"]+=ag
      teams[away]["goals_against"]+=hg

      if hg>ag:
        teams[home]["points"]+=3
        teams[home]["wins"]+=1
        teams[away]["losses"]+=1
      elif hg<ag:
        teams[away]["points"]+=3
        teams[away]["wins"]+=1
        teams[home]["losses"]+=1
      else:
        teams[home]["draws"]+=1
        teams[home]["points"]+=1
        teams[away]["draws"]+=1
        teams[away]["points"]+=1
  data=[]
  for team,stats in teams.items():
     goal_diff=stats['goals_for']-stats['goals_against']
     data.append({
         "team":team,
         "points":stats['points'],
         "wins":stats["wins"],
         "losses":stats["losses"],
         "draws":stats["draws"],
         "goals_for":stats["goals_for"],
         "goals_against":stats["goals_against"],
         "goal_diff":goal_diff
     })
  summary=pd.DataFrame(data)
  sorted_summary=summary.sort_values(["wins","goals_for","goal_diff","points"],ascending=[False,False,False,False]).reset_index(drop=True)
  sorted_summary["position"]=sorted_summary.index+1
  return sorted_summary



In [209]:
season_summaries:Dict[str,pd.DataFrame]={}
season_list: List[str] = [
    season for season, _ in season_matches if season != '2023-24'
]
for season,matches in season_matches:
  season_summary=summarise_season(matches)
  season_summaries[season]=season_summary


In [210]:
# preparing training data
X_train=[]
y_train=[]
for i in range(len(season_list)-1):
  prev_season=season_summaries[season_list[i]].copy().set_index('team')
  current_season=season_summaries[season_list[i+1]].copy().set_index('team')

  bottom_three_stats=prev_season.sort_values(["position"],ascending=[False]).head(3)
  default_features=bottom_three_stats.mean().to_dict()

  features=["points","wins","losses","draws","goals_for","goals_against","goal_diff"]
  features_rows=[]
  target_rows=[]
  for team,stats in current_season.iterrows():
    if team in prev_season.index:
       feats=prev_season.loc[team][features].to_dict()
    else:
       feats={k:default_features[k] for k in features}
    features_rows.append(feats)
    target_rows.append(stats['position'])
X_train=pd.DataFrame(features_rows)
y_train=pd.Series(target_rows)

In [211]:
# preparing latest season stats for prediction
latest_season_summary=season_summaries[season_list[-2]].copy().set_index('team')
bottom_three_stats=latest_season_summary.sort_values(["position"],ascending=[False]).head(3)
default_features=bottom_three_stats.mean().to_dict()
staying_teams=latest_season_summary.sort_values(["position"],ascending=[True]).head(16)

promoted_teams=["Almeria","Las Palmas","Burgos"]
latest_season_features=[]
for team in staying_teams.index:
  feats=latest_season_summary.loc[team][features].to_dict()
  latest_season_features.append((team,feats))
for team in promoted_teams:
  feats={k:default_features[k] for k in features}
  latest_season_features.append((team,feats))

latest_season_features=pd.DataFrame([stats for _,stats in latest_season_features],
                                    index=[team for team,_ in latest_season_features])




In [215]:
latest_season_features

,points,wins,losses,draws,goals_for,goals_against,goal_diff
Real Madrid,86.000000,26.0,4.000000,8.000000,80.0,31.000000,49.000000
Barcelona,73.000000,21.0,7.000000,10.000000,68.0,38.000000,30.000000
Ath Madrid,71.000000,21.0,9.000000,8.000000,65.0,43.000000,22.000000
Betis,65.000000,19.0,11.000000,8.000000,62.0,40.000000,22.000000
Sevilla,70.000000,18.0,4.000000,16.000000,53.0,30.000000,23.000000
Sociedad,62.000000,17.0,10.000000,11.000000,40.0,37.000000,3.000000
Villarreal,59.000000,16.0,11.000000,11.000000,63.0,37.000000,26.000000
Ath Bilbao,55.000000,14.0,11.000000,13.000000,43.0,36.000000,7.000000
Celta,46.000000,12.0,16.000000,10.000000,43.0,43.000000,0.000000
Osasuna,47.000000,12.0,15.000000,11.000000,37.0,51.000000,-14.000000


In [212]:
# building pipeling for random forest model
model=Pipeline([
    ('scaler',StandardScaler()),
    ('rf',RandomForestClassifier(
        n_estimators=100,
        max_depth=9,
        class_weight='balanced',
        random_state=42
    ))
])
model.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(class_weight='balanced', max_depth=9,
                                        random_state=42))])

In [214]:
probas=model.predict_proba(latest_season_features)
classes=model.named_steps['rf'].classes_
exp_positions=probas.dot(classes)
predicted_table=pd.DataFrame({"team":latest_season_features.index,
                              "position":exp_positions})
predicted_table=predicted_table.sort_values(['position'],ascending=[True]).reset_index(drop=True)
predicted_table

,team,position
0,Barcelona,2.830000
1,Real Madrid,3.340000
2,Ath Madrid,3.635000
3,Sociedad,5.120000
4,Villarreal,5.530000
5,Betis,5.765000
6,Ath Bilbao,8.440000
7,Sevilla,9.440000
8,Osasuna,9.910000
9,Mallorca,10.927500
